# 학습된 Checkpoint를 저장하고 불러온 후, 활용

 - graph를 freeze 한 후, 이용하기 위한 방법을 기술
 - 그래프를 freeze 할 때, graph_def ProtoBuf 형식으로 저장하였음
 - Python으로 로드하기 위해서는, graph_def ProtoBuf를 Import 하고, graph_def를 Graph로 로드해야함


In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.python.framework.graph_util import convert_variables_to_constants

/Users/sinseongjin/tf17_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [21]:
# checkpoint 파일을 확인한다
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
import os

checkpoint_path = os.path.join('./checkpoint/cnn_classifier/cnn/', 'model.ckpt-101003')

In [22]:
print_tensors_in_checkpoint_file(file_name=checkpoint_path, tensor_name='',
                                 all_tensors=True, all_tensor_names=True)

tensor_name:  EmbedSequence/embeddings
[[ 0.38603377 -0.42942917  0.05763618 ... -0.40680614  0.18765302
   0.7836783 ]
 [-0.35736287  0.6489382  -0.08572811 ... -0.0209272   0.41919473
   0.39670685]
 [-0.13057673  0.24694319  0.38319513 ...  0.25521874  0.3772233
   0.12715617]
 ...
 [ 0.09250975 -0.9849076  -0.26408935 ... -0.45885515 -0.18403554
  -0.9008472 ]
 [-0.54410625  0.3857746  -0.8793206  ...  0.52105     0.0951674
   0.19388437]
 [ 0.12058043  0.7990172  -0.87861943 ... -0.2987485  -0.3171854
   0.5782049 ]]
tensor_name:  EmbedSequence/embeddings/Adam
[[ 2.5643166e-07  3.0623505e-07 -7.2867408e-07 ...  1.0497312e-06
   1.9490201e-06  5.7679875e-07]
 [ 5.3922213e-07  1.9810336e-06  2.9101182e-07 ... -1.0345823e-06
  -4.2191499e-07  7.1524697e-09]
 [-1.1067725e-07  2.0210177e-07  1.2772237e-07 ...  9.6725319e-07
   4.6507967e-08  1.2554974e-06]
 ...
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000

In [23]:
#load all saved variables

saver = tf.train.import_meta_graph('./checkpoint/cnn_classifier/cnn/model.ckpt-101003.meta')
imported_graph = tf.get_default_graph()
graph_op = imported_graph.get_operations()
with open('output.txt', 'w') as f:
    for i in graph_op:
        f.write(str(i))
        
#output nodes: head/predictions/logistic

In [38]:
# imported_graph.get_name_scope()

#각각 opearation 불러오기
for op in imported_graph.get_operations():
    print(op.name)

global_step/Initializer/zeros
global_step
global_step/Assign
global_step/read
IsVariableInitialized
cond/Switch
cond/switch_t
cond/switch_f
cond/pred_id
cond/read/Switch
cond/read
cond/Switch_1
cond/Merge
add/y
add
tensors/component_0
tensors/component_1
tensors/component_2
buffer_size
seed
seed2
count
OneShotIterator
IteratorGetNext
EmbedSequence/embeddings/Initializer/random_uniform/shape
EmbedSequence/embeddings/Initializer/random_uniform/min
EmbedSequence/embeddings/Initializer/random_uniform/max
EmbedSequence/embeddings/Initializer/random_uniform/RandomUniform
EmbedSequence/embeddings/Initializer/random_uniform/sub
EmbedSequence/embeddings/Initializer/random_uniform/mul
EmbedSequence/embeddings/Initializer/random_uniform
EmbedSequence/embeddings
EmbedSequence/embeddings/Assign
EmbedSequence/embeddings/read
EmbedSequence/embedding_lookup
dropout/dropout/keep_prob
dropout/dropout/Shape
dropout/dropout/random_uniform/min
dropout/dropout/random_uniform/max
dropout/dropout/random_unifo

In [51]:
x = imported_graph.get_tensor_by_name('EmbedSequence/embedding_lookup:0')
y = imported_graph.get_tensor_by_name('head/predictions/logistic:0')

In [62]:
sample_db = np.random.randint(100,size=(200,50))
print(sample_db.shape)

(200, 50)


In [63]:
with tf.Session(graph=imported_graph) as sess:
    # Note: we don't nee to initialize/restore anything
    # 그래프에 Variables이 없고, only 하드코딩 숫자들만 존재
    y_out = sess.run(y, feed_dict={
        x: [sample_db] # < 45
    })
    # 이 뉴럴넷은 45보다 크기가 클 경우에, 인식하게 만들었다.
    # 그 외는 False
    print(y_out) # [[ False ]] Yay, it works!

FailedPreconditionError: Attempting to use uninitialized value conv1d/bias
	 [[Node: conv1d/bias/read = Identity[T=DT_FLOAT, _class=["loc:@conv1d/bias"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](conv1d/bias)]]

Caused by op 'conv1d/bias/read', defined at:
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sinseongjin/tf17_py3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sinseongjin/tf17_py3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/sinseongjin/tf17_py3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop

# 좀 더 쉬운 예제

In [33]:
def load_graph(frozen_graph_filename):
    
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        #protobuf file을 불러오고, parse 한다
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
#     output_graph_def = tf.graph_util.convert_variables_to_constants(
#                 sess,
#                 tf.get_default_graph().as_graph_def(),
#                 output_node_names.split(","),
#                 variable_names_blacklist=['global_step'])
        
        
    with tf.Graph().as_default() as graph:
#         tf.import_graph_def(graph_def,
#                            input_map=None,
#                            return_elements=None,
#                            op_dict=None,
#                            producer_op_list=None
#                            name='')
        tf.import_graph_def(graph_def, name='prefix')
        
    return graph

graph = load_graph('./results/frozen_model.pb')

In [4]:
#각각 opearation 불러오기
for op in graph.get_operations():
    print(op.name)

prefix/Placeholder/inputs_placeholder
prefix/NN/W1
prefix/NN/W1/read
prefix/NN/b1
prefix/NN/b1/read
prefix/NN/W2
prefix/NN/W2/read
prefix/NN/b2
prefix/NN/b2/read
prefix/NN/MatMul
prefix/NN/add
prefix/NN/Relu
prefix/NN/MatMul_1
prefix/NN/add_1
prefix/NN/Relu_1
prefix/NN/Add
prefix/NN/truediv/y
prefix/NN/truediv
prefix/Accuracy/predictions/y
prefix/Accuracy/predictions


In [5]:
# input과 output nodes 접근
x = graph.get_tensor_by_name('prefix/Placeholder/inputs_placeholder:0')
y = graph.get_tensor_by_name('prefix/Accuracy/predictions:0')

In [9]:
with tf.Session(graph=graph) as sess:
    # Note: we don't nee to initialize/restore anything
    # 그래프에 Variables이 없고, only 하드코딩 숫자들만 존재
    y_out = sess.run(y, feed_dict={
        x: [[3, 5, 7, 4, 5, 1, 1, 1, 1, 1]] # < 45
    })
    # 이 뉴럴넷은 45보다 크기가 클 경우에, 인식하게 만들었다.
    # 그 외는 False
    print(y_out) # [[ False ]] Yay, it works!

[[False]]


# Appendix

https://blog.metaflow.fr/tensorflow-how-to-freeze-a-model-and-serve-it-with-a-python-api-d4f3596b3adc

https://blog.metaflow.fr/tensorflow-saving-restoring-and-mixing-multiple-models-c4c94d5d7125

http://cv-tricks.com/how-to/freeze-tensorflow-models/